In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [10]:
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [4]:
file_path = "ds3_cleaned.csv"
ds3 = pd.read_csv(file_path, index_col=0)

In [5]:
ds3.head()

,pha,sigma_e,sigma_a,sigma_q,sigma_i,sigma_om,sigma_w,sigma_ma,sigma_ad,sigma_n,sigma_tp,sigma_per
spkid,,,,,,,,,,,,
2000433,0,9.400000e-09,1.600000e-10,1.400000e-08,1.200000e-06,0.000004,0.000004,0.000001,1.900000e-10,9.100000e-11,0.000003,1.000000e-07
2000719,0,2.200000e-08,2.500000e-09,5.800000e-08,2.900000e-06,0.000016,0.000016,0.000003,3.900000e-09,3.300000e-10,0.000012,2.300000e-06
2000887,0,2.700000e-08,3.000000e-09,6.700000e-08,3.100000e-06,0.000023,0.000024,0.000004,4.700000e-09,4.600000e-10,0.000014,2.600000e-06
2001036,0,1.100000e-08,1.900000e-09,3.000000e-08,4.900000e-07,0.000001,0.000001,0.000002,2.900000e-09,2.400000e-10,0.000006,1.700000e-06
2001221,0,3.300000e-08,9.200000e-10,6.300000e-08,4.400000e-06,0.000021,0.000022,0.000008,1.300000e-09,2.700000e-10,0.000021,7.000000e-07


In [6]:
# Will use standard scaler to try and reduce bias.  Will drop pha and make it y

# Remove pha status target from features data
y = ds3.pha
X = ds3.drop(columns=['pha'])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.923


In [11]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, y_pred)

In [12]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_pred))


Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,6380,24
Actual 1,515,38


Accuracy Score : 0.9225240764697427
Classification Report
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      6404
           1       0.61      0.07      0.12       553

    accuracy                           0.92      6957
   macro avg       0.77      0.53      0.54      6957
weighted avg       0.90      0.92      0.89      6957



In [13]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.09837934, 0.03321166, 0.13933137, 0.14642164, 0.12414508,
       0.10623009, 0.12004934, 0.0344859 , 0.06091186, 0.1113595 ,
       0.02547421])

In [14]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.1464216411346469, 'sigma_i'),
 (0.1393313675394061, 'sigma_q'),
 (0.12414507906089173, 'sigma_om'),
 (0.1200493443592926, 'sigma_ma'),
 (0.11135950006250393, 'sigma_tp'),
 (0.10623008895216383, 'sigma_w'),
 (0.09837934186106456, 'sigma_e'),
 (0.06091186377348619, 'sigma_n'),
 (0.03448590004524001, 'sigma_ad'),
 (0.03321166238233031, 'sigma_a'),
 (0.02547421082897386, 'sigma_per')]